In [ ]:
import pandas as pd
import numpy as np


rfq = pd.read_csv('https://raw.githubusercontent.com/LarsBryld/auctions/master/bfm492D.csv', sep=',', parse_dates=True, dayfirst=False, skiprows=2)


In [ ]:
# Creating 
rfq['Second'] = pd.DatetimeIndex(rfq['Exec Time']).second
rfq['Minute'] = pd.DatetimeIndex(rfq['Exec Time']).minute
rfq['Hour'] = pd.DatetimeIndex(rfq['Exec Time']).hour
rfq['Day'] = pd.DatetimeIndex(rfq['Trade Dt']).day
rfq['Month'] = pd.DatetimeIndex(rfq['Trade Dt']).month
rfq['Year'] = pd.DatetimeIndex(rfq['Trade Dt']).year  
rfq['Source'] = 'RFQ'

# Creating date variable used for plotting
rfq['Date'] = pd.to_datetime(rfq['Trade Dt'], format='%d/%m/%Y')
rfq['WeekDay'] = rfq['Date'].dt.day_name()
rfq['MonthName'] = rfq['Date'].dt.month_name()

#Calculating days to maturity and rounding result 
rfq['Maturity'] = (pd.to_datetime(rfq['Mat Dt'], format='%d/%m/%Y') - rfq['Date']) / np.timedelta64(1,'D') / 365
rfq['Maturity'] = rfq['Maturity'].round(1)

import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'colab'

#import plotly.graph_objects as go
#fig = go.Figure(data=go.Bar(y=[2, 3, 1]))
#fig.show()


#Create inverted price measure 
rfq.loc[rfq['Side'] == 'B', 'PriceDiff'] = (rfq['CBBT Mid Prc'] - rfq['Price']) 
rfq.loc[rfq['Side'] == 'S', 'PriceDiff'] = (rfq['Price'] - rfq['CBBT Mid Prc'])


# Create inverted price dummy 
rfq.loc[rfq['PriceDiff'] < 0, 'PriceDiffDummy'] = 'Inverted'
rfq.loc[rfq['PriceDiff'] > 0, 'PriceDiffDummy'] = 'Non-Inverted'
rfq.loc[rfq['PriceDiff'] == 0, 'PriceDiffDummy'] = 'Non-Inverted'


# away from mid volume  
rfq.loc[rfq['PriceDiffDummy'] == 1, 'VolumenAwayFromMid'] = (rfq['Quantity'] * rfq['PriceDiff'] * 1)
rfq.loc[rfq['PriceDiffDummy'] == 0, 'VolumenAwayFromMid'] = (rfq['Quantity'] * rfq['PriceDiff'] * 1)

rfq.loc[rfq['Status'].isin(['AllocSent', 'Accepted', 'AllocSave', 'AllocAcpt']), 'Status2'] = ('Accepted')
rfq.loc[rfq['Status'].isin(['Tie-TW', 'Covered']), 'Status2'] = ('Covered')
rfq.loc[rfq['Status'].isin(['TrAway', 'Passed', 'C/Expire', 'D/Expire', 'Rejected', 'Corrected', 'Cancel', 'C/Cancel', 'AllocCanc']), 'Status2'] = ('TrAway')


rfq = rfq[rfq['TS Book Name'].isin(['STAT', 'FLEX', 'REAL', 'ILLIKVID', 'FLOAT', 'INFLATIO'])]

In [ ]:
#removing Perp bonds from Data set
rfq = rfq[rfq['Maturity'] < 100]

#removing Perp bonds from Data set
rfq = rfq[rfq['Yield'] < 100 ]

#removing bonds from Data set with price > 200 
rfq = rfq[rfq['Price'] < 200 ]

rfq = rfq[rfq['PriceDiff'] < 9 ]


#Reducing the rfq df to be of one year dimension
rfq = rfq[rfq['Date']> max(rfq['Date']) - pd.DateOffset(years=1)]


#changing nan in dls comp to zeros
rfq['DlrsCmp'].fillna(0, inplace=True)

real = rfq[rfq['TS Book Name'] == 'REAL']

Latest date in main RFQ data set is 04-Jan-2021. Earliest date is 06-Jan-2020


In [ ]:
mix = real[real['Customer'].isin(['W&W ASSET MANAGEMENT GMBH', 'MITSUBISHI UFJ KOKU AST MGT CO LTD'])]


fig = px.bar(mix, x="WeekDay", y="Quantity", color="Side", title="Day of Week <br>RFQ's from W&W ASSET MANAGEMENT", facet_col="Customer", facet_col_wrap=6,)
fig.update_xaxes(categoryorder='array')
fig.update_xaxes(categoryarray= ['Monday','Tuesday','Wednesday','Thursday', 'Friday', 'December'])
#fig.update_yaxes(tick0=1, range=[0, 3500000000],dtick=500000000)
fig.show()
